# [Scope](https://docs.julialang.org/en/v1/manual/variables-and-scoping/)

Julia uses _**lexical scoping**_:
* a function's scope does not inherit from its caller's scope, but from the scope in which the function was defined.
* the scope of variables can be _**inferred from the source code alone**_.

[Scope constructs in Julia](https://docs.julialang.org/en/v1/manual/variables-and-scoping/#man-scope-table-1)

In [1]:
module MyModule
x = -1
foo() = x
end

Main.MyModule

In [2]:
import .MyModule

x = 5
MyModule.foo()  # foo's scope inherits from the global scope where it is defined
                # instead of from the caller

-1

## Global scope

* `module` introduces a new global scope which is a so-called _**namespace**_
* there is no all-encompassing global scope. 
* Note that _**variable bindings can only be changed within their global scope and not from an outside module**_.
    * _**Global assignment is always module-local**_.

Type and macro definitions as they can _**only**_ appear at the global scope. 

In [3]:
function func1()
    struct A
        x
    end
end

ErrorException: syntax: "struct" expression not at top level

## Local scope

* A local scope inherits:
    1. all the variables from a parent local scope, both for reading and writing.
    1. all global variables that are assigned in its parent global scope block.
* A newly introduced variable in a local scope does not back-propagate to its parent scope.

In a local scope, all variables are inherited from its parent global scope block unless:
1. an assignment would result in a modified global variable.
    * In a local scope, global variables are _**only inherited for reading, not for writing**_.
1. a variable is specifically marked with the keyword `local`.

In [4]:
x = 5  # x in the global scope.

function func2()
    x = 1  # this assigment creates a new local variable
    function func3()  # nested function can modify variables in parent scope.
        x += 5  # Access to x in the parent scope and modifiy it.
    end
    
    x + func3()
end

@show func2()
x  # global x does not change.

func2() = 7


5

## `let` blocks

`let` statements allocate new variable bindings each time they run. 

In [5]:
x = 5

# let is evaluated in the global scope
let x = x, z  # introduce new local variable x and z into a local scope 
    x += 1    # assign the value of global x to local x and change the value of local x
    @show x   # show the value of local variable
end

@show x  # global x does not change.

x = 6
x = 5


5

A for loop or comprehension iteration variable is always a new variable, but the keyword `outer` canbe used to reuse the iteration variable.

In [6]:
i = 0

for i = 1 : 3 end # this i is a new variable
@show i

i = 0


0

In [7]:
function func4()
    i = 0
    # outer can only be used in a local scope.
    for outer i = 1 : 3 end  # reuse previous defined i
    @show i
end

func4()

i = 3


3

### Keyword `local` and `global`

_**Inside a local scope**_, a variable can be forced to be a new local variable using keyword `local`.

In [8]:
function func5()
    x = 5
    
    for i = 5 : 7
        local x = i + 1  # x is a new variable.
        @show x
    end
    
    @show x  # x in line 2 does not change.
end

func5();

x = 6
x = 7
x = 8
x = 5


_**Inside a local scope**_, a global variable can be assigned to by using the keyword `global`.

In [9]:
a = 5

function func6()
    @show a + 1  # this is OK.
                 # the local scope inherits global variables from the global scope where it defines.
end

func6()
@show a

a + 1 = 6
a = 5


5

In [10]:
function func7()
    @show a = a + 1  # this will raise an error.
                     # in local variable, assignment will create a new variable binding!
                     # the variable binding in outer global scope cannot be changed inside a local scope.
end
func7()

UndefVarError: UndefVarError: a not defined

In [13]:
b = 5
function func8()
    global b = b + 1  # a global variable canbe assigned to using keyword `global`.
end

func8()
@show b;  # b's value is changed in the local scope.

b = 6
